In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
data = pd.read_csv(r"C:\Users\ryanp\Desktop\Springboard\Springboard\E-commerce data\data.csv", encoding= 'unicode_escape')
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


# Data Wrangling

In [3]:
data.shape

(541909, 8)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


**The data contains 541,909 columns and 8 rows. We can also see the different types of data contained within the columns.**

In [5]:
missing = pd.concat([data.isnull().sum(), 100 * data.isnull().mean()], axis=1)
missing.columns = ['count', '%']
missing.sort_values(by='count')

,count,%
InvoiceNo,0,0.000000
StockCode,0,0.000000
Quantity,0,0.000000
InvoiceDate,0,0.000000
UnitPrice,0,0.000000
Country,0,0.000000
Description,1454,0.268311
CustomerID,135080,24.926694


In [6]:
data = data.dropna()
data.shape

(406829, 8)

**There is about 25% total data that is missing, with the majority of missing data within the Customer ID column. Without the Customer ID, it is difficult to fill that value with any other data and to avoid data manipulation, all rows with missing data will be dropped. The new data shape can be seen.** 

In [7]:
data.describe()

,Quantity,UnitPrice,CustomerID
count,406829.000000,406829.000000,406829.000000
mean,12.061303,3.460471,15287.690570
std,248.693370,69.315162,1713.600303
min,-80995.000000,0.000000,12346.000000
25%,2.000000,1.250000,13953.000000
50%,5.000000,1.950000,15152.000000
75%,12.000000,3.750000,16791.000000
max,80995.000000,38970.000000,18287.000000


**The data contains outliers as you can seen within the Quantity column containing negative values. Further outliers will be investigated.**

In [8]:
data[data['InvoiceNo'] == 0].count()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [9]:
data[data['Quantity'] < 0].count()

InvoiceNo      8905
StockCode      8905
Description    8905
Quantity       8905
InvoiceDate    8905
UnitPrice      8905
CustomerID     8905
Country        8905
dtype: int64

In [10]:
data = data[data['Quantity'] >= 0]
data.shape

(397924, 8)

In [11]:
data[data['UnitPrice'] < 0].count()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [12]:
data[data['UnitPrice'] == 0].count()

InvoiceNo      40
StockCode      40
Description    40
Quantity       40
InvoiceDate    40
UnitPrice      40
CustomerID     40
Country        40
dtype: int64

In [13]:
data = data[data['UnitPrice'] > 0]
data.shape

(397884, 8)

In [14]:
data.duplicated().sum()

5192

In [15]:
data.drop_duplicates(inplace=True)
data.shape

(392692, 8)

**There were more outliers identified within the Unit Price column and duplicated rows were identified. All these rows will be dropped.**

In [16]:
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])
data['InvoiceDate'].head()

0   2010-12-01 08:26:00
1   2010-12-01 08:26:00
2   2010-12-01 08:26:00
3   2010-12-01 08:26:00
4   2010-12-01 08:26:00
Name: InvoiceDate, dtype: datetime64[ns]

**The Invoice Date was changed to represent its true value as a date.**

In [17]:
data['Total Cost'] = data["Quantity"] * data["UnitPrice"]

In [18]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Total Cost
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


**A new column called Total Cost was added to the DataFrame. This data contains the unit price mulitplied by quantity to find how much each order costed for each customer.**

In [19]:
data = data[['InvoiceNo', 'InvoiceDate', 'CustomerID', 'Country', 'StockCode', 'Description', 'Quantity', 'UnitPrice', 'Total Cost']]

In [20]:
data.head()

,InvoiceNo,InvoiceDate,CustomerID,Country,StockCode,Description,Quantity,UnitPrice,Total Cost
0,536365,2010-12-01 08:26:00,17850.0,United Kingdom,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2.55,15.30
1,536365,2010-12-01 08:26:00,17850.0,United Kingdom,71053,WHITE METAL LANTERN,6,3.39,20.34
2,536365,2010-12-01 08:26:00,17850.0,United Kingdom,84406B,CREAM CUPID HEARTS COAT HANGER,8,2.75,22.00
3,536365,2010-12-01 08:26:00,17850.0,United Kingdom,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,3.39,20.34
4,536365,2010-12-01 08:26:00,17850.0,United Kingdom,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,3.39,20.34


**The DataFrame was organized to be read easier.**

In [21]:
data['CustomerID'] = data['CustomerID'].astype(int)
data['CustomerID'].head(1)

0    17850
Name: CustomerID, dtype: int32

In [22]:
data.describe()

,CustomerID,Quantity,UnitPrice,Total Cost
count,392692.000000,392692.000000,392692.000000,392692.000000
mean,15287.843865,13.119702,3.125914,22.631500
std,1713.539549,180.492832,22.241836,311.099224
min,12346.000000,1.000000,0.001000,0.001000
25%,13955.000000,2.000000,1.250000,4.950000
50%,15150.000000,6.000000,1.950000,12.450000
75%,16791.000000,12.000000,3.750000,19.800000
max,18287.000000,80995.000000,8142.750000,168469.600000


**The numerical data has much improved and appears to be ready to be explored and analyzed.**

In [23]:
data['InvoiceNo'].nunique()

18532

In [27]:
data['StockCode'].nunique()

3665

In [25]:
data['CustomerID'].nunique()

4338

In [26]:
data['Country'].nunique()

37

In [28]:
data['Description'].nunique()

3877